# Modelo de programación lineal

In [1]:
import pandas as pd
import gurobipy as grb

## Entrada de datos: Instancia 4x50x1.5x1

In [2]:
datafile="4x50x1.5x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

## Modelo

In [3]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Academic license - for non-commercial use only - expires 2021-06-11
Using license file C:\Users\elPatan03\gurobi.lic
Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x499e0437
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 11.0000000
Presolve removed 186704 rows and 29004 columns
Presolve time: 1.16s
Presolved: 746 rows, 996 columns, 13054 nonzeros
Found heuristic solution: objective 37.0000000
Variable types: 0 continuous, 996 integer (996 binary)

Root relaxation: objectiv

## Entrada de datos: Instancia 4x50x1.5x2

In [4]:
datafile="4x50x1.5x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [5]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0xad7b74f4
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 186552 rows and 28972 columns
Presolve time: 1.22s
Presolved: 898 rows, 1028 columns, 15308 nonzeros
Variable types: 0 continuous, 1028 integer (1028 binary)

Root relaxation: objective 4.200000e+01, 438 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumb

## Entrada de datos: Instancia 4x50x1.5x3

In [6]:
datafile="4x50x1.5x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [7]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x036ec0ff
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 186624 rows and 29008 columns
Presolve time: 0.98s
Presolved: 826 rows, 992 columns, 13691 nonzeros
Variable types: 0 continuous, 992 integer (992 binary)

Root relaxation: objective 4.567139e+01, 682 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

## Entrada de datos: Instancia 4x50x1.5x4

In [8]:
datafile="4x50x1.5x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [9]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x2fa90850
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 186688 rows and 29020 columns
Presolve time: 0.95s
Presolved: 762 rows, 980 columns, 13448 nonzeros
Variable types: 0 continuous, 980 integer (980 binary)

Root relaxation: objective 4.300000e+01, 416 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

## Entrada de datos: Instancia 4x50x1.5x5

In [10]:
datafile="4x50x1.5x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [11]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x9cce7c3f
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 186578 rows and 28984 columns
Presolve time: 0.87s
Presolved: 872 rows, 1016 columns, 13309 nonzeros
Variable types: 0 continuous, 1016 integer (1016 binary)

Root relaxation: objective 4.300000e+01, 421 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumb

## Entrada de datos: Instancia 4x50x2x1

In [12]:
datafile="4x50x2x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [13]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x85e054ef
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 186795 rows and 29144 columns
Presolve time: 0.83s
Presolved: 655 rows, 856 columns, 10545 nonzeros
Variable types: 0 continuous, 856 integer (856 binary)

Root relaxation: objective 4.254480e+01, 406 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

## Entrada de datos: Instancia 4x50x2x2

In [14]:
datafile="4x50x2x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [15]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x8d839766
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 11.0000000
Presolve removed 186551 rows and 29076 columns
Presolve time: 0.83s
Presolved: 899 rows, 924 columns, 13611 nonzeros
Variable types: 0 continuous, 924 integer (924 binary)

Root relaxation: objective 4.094477e+01, 981 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

## Entrada de datos: Instancia 4x50x2x3

In [16]:
datafile="4x50x2x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [17]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x15609ebd
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 186609 rows and 29068 columns
Presolve time: 0.86s
Presolved: 841 rows, 932 columns, 13600 nonzeros
Variable types: 0 continuous, 932 integer (932 binary)

Root relaxation: objective 4.300000e+01, 334 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

## Entrada de datos: Instancia 4x50x2x4

In [18]:
datafile="4x50x2x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [19]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x926a867e
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 186476 rows and 28956 columns
Presolve time: 1.31s
Presolved: 974 rows, 1044 columns, 17902 nonzeros
Variable types: 0 continuous, 1044 integer (1044 binary)

Root relaxation: objective 4.400000e+01, 721 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumb

## Entrada de datos: Instancia 4x50x2x5

In [20]:
datafile="4x50x2x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [21]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x08f4715e
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 186889 rows and 29328 columns
Presolve time: 0.79s
Presolved: 561 rows, 672 columns, 8241 nonzeros
Found heuristic solution: objective 37.0000000
Variable types: 0 continuous, 672 integer (672 binary)

Root relaxation: objective 4.100000e+01, 360 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work

## Entrada de datos: Instancia 4x50x2.5x1

In [22]:
datafile="4x50x2.5x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [23]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x19ec90e0
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 186657 rows and 29332 columns
Presolve time: 0.62s
Presolved: 793 rows, 668 columns, 9828 nonzeros
Variable types: 0 continuous, 668 integer (668 binary)

Root relaxation: objective 4.064973e+01, 452 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

## Entrada de datos: Instancia 4x50x2.5x2

In [24]:
datafile="4x50x2.5x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [25]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0xf29c8901
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 15.0000000
Presolve removed 186700 rows and 29212 columns
Presolve time: 0.73s
Presolved: 750 rows, 788 columns, 10785 nonzeros
Variable types: 0 continuous, 788 integer (788 binary)

Root relaxation: objective 3.900000e+01, 302 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

## Entrada de datos: Instancia 4x50x2.5x3

In [26]:
datafile="4x50x2.5x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [27]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x146b9bc2
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 11.0000000
Presolve removed 186862 rows and 29244 columns
Presolve time: 0.85s
Presolved: 588 rows, 756 columns, 11594 nonzeros
Found heuristic solution: objective 32.0000000
Variable types: 0 continuous, 756 integer (756 binary)

Root relaxation: objective 3.792915e+01, 830 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Wor

## Entrada de datos: Instancia 4x50x2.5x4

In [28]:
datafile="4x50x2.5x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [29]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x34995579
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 16.0000000
Presolve removed 186888 rows and 29384 columns
Presolve time: 0.71s
Presolved: 562 rows, 616 columns, 7924 nonzeros
Variable types: 0 continuous, 616 integer (616 binary)

Root relaxation: objective 3.600000e+01, 252 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

## Entrada de datos: Instancia 4x50x2.5x5

In [30]:
datafile="4x50x2.5x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [31]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x5ca31d71
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 186838 rows and 29372 columns
Presolve time: 0.71s
Presolved: 612 rows, 628 columns, 8852 nonzeros
Found heuristic solution: objective 29.0000000
Variable types: 0 continuous, 628 integer (628 binary)

Root relaxation: objective 3.758162e+01, 408 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work

## Entrada de datos: Instancia 4x50x3x1

In [32]:
datafile="4x50x3x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [33]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x725792cd
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 16.0000000
Presolve removed 187068 rows and 29476 columns
Presolve time: 0.54s
Presolved: 382 rows, 524 columns, 5690 nonzeros
Variable types: 0 continuous, 524 integer (524 binary)

Root relaxation: objective 3.677415e+01, 323 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

## Entrada de datos: Instancia 4x50x3x2

In [34]:
datafile="4x50x3x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [35]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0xcdfde65c
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 10.0000000
Presolve removed 186959 rows and 29384 columns
Presolve time: 0.85s
Presolved: 491 rows, 616 columns, 8148 nonzeros
Found heuristic solution: objective 32.0000000
Variable types: 0 continuous, 616 integer (616 binary)

Root relaxation: objective 3.438217e+01, 528 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work

## Entrada de datos: Instancia 4x50x3x3

In [36]:
datafile="4x50x3x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [37]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0xf864b99e
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 186929 rows and 29464 columns
Presolve time: 0.68s
Presolved: 521 rows, 536 columns, 7180 nonzeros
Variable types: 0 continuous, 536 integer (536 binary)

Root relaxation: objective 3.452681e+01, 434 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

## Entrada de datos: Instancia 4x50x3x4

In [38]:
datafile="4x50x3x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [39]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0xe9917700
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 15.0000000
Presolve removed 186735 rows and 29353 columns
Presolve time: 0.66s
Presolved: 715 rows, 647 columns, 10108 nonzeros
Found heuristic solution: objective 33.0000000
Variable types: 0 continuous, 647 integer (647 binary)

Root relaxation: objective 3.916766e+01, 1073 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Wo

## Entrada de datos: Instancia 4x50x3x5

In [40]:
datafile="4x50x3x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [41]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x32d7995b
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 15.0000000
Presolve removed 186946 rows and 29384 columns
Presolve time: 0.63s
Presolved: 504 rows, 616 columns, 7688 nonzeros
Variable types: 0 continuous, 616 integer (616 binary)

Root relaxation: objective 3.700000e+01, 426 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

## Entrada de datos: Instancia 4x50x3.5x1

In [42]:
datafile="4x50x3.5x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [43]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x2ad9b4bd
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 186688 rows and 29440 columns
Presolve time: 0.81s
Presolved: 762 rows, 560 columns, 9256 nonzeros
Variable types: 0 continuous, 560 integer (560 binary)

Root relaxation: objective 3.322338e+01, 539 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

## Entrada de datos: Instancia 4x50x3.5x2

In [44]:
datafile="4x50x3.5x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [45]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x8d705b4e
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 186649 rows and 29396 columns
Presolve time: 0.71s
Presolved: 801 rows, 604 columns, 10276 nonzeros
Variable types: 0 continuous, 604 integer (604 binary)

Root relaxation: objective 3.269965e+01, 485 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

## Entrada de datos: Instancia 4x50x3.5x3

In [46]:
datafile="4x50x3.5x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [47]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x6a491839
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 186927 rows and 29428 columns
Presolve time: 0.62s
Presolved: 523 rows, 572 columns, 7100 nonzeros
Variable types: 0 continuous, 572 integer (572 binary)

Root relaxation: objective 3.389187e+01, 323 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

## Entrada de datos: Instancia 4x50x3.5x4

In [48]:
datafile="4x50x3.5x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [49]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x52080db6
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 186989 rows and 29472 columns
Presolve time: 0.50s
Presolved: 461 rows, 528 columns, 7104 nonzeros
Variable types: 0 continuous, 528 integer (528 binary)

Root relaxation: objective 3.207033e+01, 223 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

## Entrada de datos: Instancia 4x50x3.5x5

In [50]:
datafile="4x50x3.5x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [51]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x821afefe
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 186908 rows and 29432 columns
Presolve time: 0.62s
Presolved: 542 rows, 568 columns, 7942 nonzeros
Variable types: 0 continuous, 568 integer (568 binary)

Root relaxation: objective 3.569115e+01, 371 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

## Entrada de datos: Instancia 4x50x4x1

In [52]:
datafile="4x50x4x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [53]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x6f9ed2c2
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 187032 rows and 29544 columns
Presolve time: 0.62s
Presolved: 418 rows, 456 columns, 5774 nonzeros
Variable types: 0 continuous, 456 integer (456 binary)

Root relaxation: objective 2.828185e+01, 307 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

## Entrada de datos: Instancia 4x50x4x2

In [54]:
datafile="4x50x4x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [55]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x8e28e4f2
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 18.0000000
Presolve removed 186585 rows and 29540 columns
Presolve time: 0.68s
Presolved: 865 rows, 460 columns, 13024 nonzeros
Variable types: 0 continuous, 460 integer (460 binary)

Root relaxation: objective 3.140924e+01, 521 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

## Entrada de datos: Instancia 4x50x4x3

In [56]:
datafile="4x50x4x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [57]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x2e80c335
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 15.0000000
Presolve removed 187137 rows and 29632 columns
Presolve time: 0.39s
Presolved: 313 rows, 368 columns, 3384 nonzeros
Variable types: 0 continuous, 368 integer (368 binary)

Root relaxation: objective 2.900000e+01, 193 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

## Entrada de datos: Instancia 4x50x4x4

In [58]:
datafile="4x50x4x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [59]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x489d7ca0
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 186785 rows and 29548 columns
Presolve time: 0.66s
Presolved: 665 rows, 452 columns, 8444 nonzeros
Found heuristic solution: objective 23.0000000
Variable types: 0 continuous, 452 integer (452 binary)

Root relaxation: objective 3.008442e+01, 439 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work

## Entrada de datos: Instancia 4x50x4x5

In [60]:
datafile="4x50x4x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [61]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x928df602
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 187165 rows and 29512 columns
Presolve time: 0.49s
Presolved: 285 rows, 488 columns, 5512 nonzeros
Variable types: 0 continuous, 488 integer (488 binary)

Root relaxation: objective 2.982273e+01, 315 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

## Entrada de datos: Instancia 4x50x4.5x1

In [62]:
datafile="4x50x4.5x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [63]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x9f88c397
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 15.0000000
Presolve removed 187093 rows and 29608 columns
Presolve time: 0.41s
Presolved: 357 rows, 392 columns, 4646 nonzeros
Variable types: 0 continuous, 392 integer (392 binary)

Root relaxation: objective 2.929184e+01, 239 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

## Entrada de datos: Instancia 4x50x4.5x2

In [64]:
datafile="4x50x4.5x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [65]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x89f8ca5c
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 187082 rows and 29612 columns
Presolve time: 0.47s
Presolved: 368 rows, 388 columns, 4668 nonzeros
Variable types: 0 continuous, 388 integer (388 binary)

Root relaxation: objective 2.747358e+01, 445 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

## Entrada de datos: Instancia 4x50x4.5x3

In [66]:
datafile="4x50x4.5x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [67]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x2d58b109
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 15.0000000
Presolve removed 187062 rows and 29628 columns
Presolve time: 0.46s
Presolved: 388 rows, 372 columns, 5124 nonzeros
Variable types: 0 continuous, 372 integer (372 binary)

Root relaxation: objective 2.818530e+01, 327 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

## Entrada de datos: Instancia 4x50x4.5x4

In [68]:
datafile="4x50x4.5x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [69]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0x4de2f815
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 187054 rows and 29640 columns
Presolve time: 0.43s
Presolved: 396 rows, 360 columns, 5261 nonzeros
Variable types: 0 continuous, 360 integer (360 binary)

Root relaxation: objective 2.635131e+01, 169 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

## Entrada de datos: Instancia 4x50x4.5x5

In [70]:
datafile="4x50x4.5x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [71]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0xc801f323
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 186906 rows and 29580 columns
Presolve time: 0.70s
Presolved: 544 rows, 420 columns, 7860 nonzeros
Found heuristic solution: objective 24.0000000
Variable types: 0 continuous, 420 integer (420 binary)

Root relaxation: objective 2.737621e+01, 378 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work

## Entrada de datos: Instancia 4x50x5x1

In [72]:
datafile="4x50x5x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [73]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0xd4041d0e
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 10.0000000
Presolve removed 186993 rows and 29572 columns
Presolve time: 0.63s
Presolved: 457 rows, 428 columns, 6808 nonzeros
Variable types: 0 continuous, 428 integer (428 binary)

Root relaxation: objective 2.473305e+01, 266 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

## Entrada de datos: Instancia 4x50x5x2

In [74]:
datafile="4x50x5x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [75]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0xd5b0e3d9
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 187180 rows and 29648 columns
Presolve time: 0.37s
Presolved: 270 rows, 352 columns, 3868 nonzeros
Variable types: 0 continuous, 352 integer (352 binary)

Root relaxation: objective 2.512139e+01, 132 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

## Entrada de datos: Instancia 4x50x5x3

In [76]:
datafile="4x50x5x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [77]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0xa791a2e9
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 186776 rows and 29520 columns
Presolve time: 0.47s
Presolved: 674 rows, 480 columns, 7432 nonzeros
Variable types: 0 continuous, 480 integer (480 binary)

Root relaxation: objective 2.764811e+01, 308 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

## Entrada de datos: Instancia 4x50x5x4

In [78]:
datafile="4x50x5x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [79]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0xed3aa28b
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 15.0000000
Presolve removed 186655 rows and 29544 columns
Presolve time: 0.66s
Presolved: 795 rows, 456 columns, 10084 nonzeros
Variable types: 0 continuous, 456 integer (456 binary)

Root relaxation: objective 2.781710e+01, 704 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

## Entrada de datos: Instancia 4x50x5x5

In [80]:
datafile="4x50x5x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

50

In [81]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 187450 rows, 30000 columns and 1188400 nonzeros
Model fingerprint: 0xf0bf762d
Variable types: 0 continuous, 30000 integer (30000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 11.0000000
Presolve removed 186742 rows and 29568 columns
Presolve time: 0.56s
Presolved: 708 rows, 432 columns, 7820 nonzeros
Variable types: 0 continuous, 432 integer (432 binary)

Root relaxation: objective 2.742992e+01, 398 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 